https://blog.51cto.com/u_15343606/6364392
macbook苹果m1芯片训练机器学习、深度学习模型,resnet101在mnist手写数字识别上做加速，torch.device("mps")

In [2]:
# 然后使用chatGPT生成一个resnet101的训练代码，这里注意，如果网络特别轻的话是没有加速效果的，还没有cpu的计算来的快
# 这里要选择好设备不是"cuda"了，cuda是nvidia深度学习加速的配置
# 设置设备
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps") #torch.device("cpu")
# resnet101的训练代码，训练mnist手写数字识别，之前我还尝试了两层linear的训练代码，低估了apple 的 torch.device("mps"),这两层linear的简单神经网络完全加速不起来，还不如torch.device("cpu")快，换成了resnet101加速效果就很明显了，目测速度在mps上比cpu快了5倍左右
 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.models import resnet101
from tqdm import tqdm

# 设置设备
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")  #torch.device("cpu")


In [3]:


# 加载 MNIST 数据集
train_dataset = MNIST(root="/Users/emma/Downloads/dataset", train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root="/Users/emma/Downloads/dataset", train=False, transform=ToTensor())


100%|██████████| 9912422/9912422 [00:04<00:00, 2350429.65it/s]


Extracting /Users/emma/Downloads/dataset/MNIST/raw/train-images-idx3-ubyte.gz to /Users/emma/Downloads/dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 500: Internal Privoxy Error



100%|██████████| 28881/28881 [00:00<00:00, 174929.23it/s]


Extracting /Users/emma/Downloads/dataset/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/emma/Downloads/dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1395179.05it/s]


Extracting /Users/emma/Downloads/dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/emma/Downloads/dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7994346.94it/s]

Extracting /Users/emma/Downloads/dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/emma/Downloads/dataset/MNIST/raw



In [4]:

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [5]:
#学习代码，查看model 各层的参数
import torch
import torchvision.models as models

model = models.resnet101(pretrained=False)
print(model)


/Users/emma/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/emma/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:

# 定义 ResNet-101 模型
model = resnet101(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(2048, 10)  # 替换最后一层全连接层
model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练和评估函数
def train(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader, desc="Training"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    return accuracy

# 训练和评估
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss = train(model, train_loader, optimizer, criterion)
    print(f"Training Loss: {train_loss:.4f}")
    
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy: {test_acc:.2f}%")



/Users/emma/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/emma/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10


Training: 100%|██████████| 938/938 [03:50<00:00,  4.06it/s]


Training Loss: 0.2976


Evaluating: 100%|██████████| 157/157 [00:09<00:00, 16.58it/s]


...

Training Loss: 0.0890


Evaluating: 100%|██████████| 157/157 [00:10<00:00, 14.34it/s]


Test Accuracy: 97.62%
Epoch 5/10


Training:  50%|████▉     | 465/938 [02:05<02:07,  3.70it/s]


KeyboardInterrupt: 